# MMSegmentation Tutorial
Welcome to MMSegmentation! 

In this tutorial, we demo
* How to do inference with MMSeg trained weight
* How to train on your own dataset and visualize the results. 

## Install MMSegmentation
This step may take several minutes. 

We use PyTorch 1.10 and CUDA 11.1 for this tutorial. You may install other versions by change the version number in pip install command. 

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
# Install PyTorch
!conda install pytorch=1.10.0 torchvision cudatoolkit=11.1 -c pytorch
# Install MMCV
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10/index.html

In [ ]:
!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

## Run Inference with MMSeg trained weight

In [ ]:
!mkdir checkpoints
!wget https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth -P checkpoints

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [ ]:
config_file = 'configs/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file = 'checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

In [ ]:
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

In [ ]:
# test a single image
img = 'demo/demo.png'
result = inference_segmentor(model, img)

In [ ]:
# show the results
show_result_pyplot(model, img, result, get_palette('cityscapes'), out_file = "./demo_infered.png")

In [ ]:
show_result_pyplot

In [ ]:
!pip install opencv-python

In [ ]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
vidcap = cv2.VideoCapture('/content/North American Car Driving.mp4')

In [ ]:
vidcap.isOpened()

In [ ]:
success,image = vidcap.read()
count = 0

In [ ]:
image.shape

In [ ]:
!mkdir frames

In [ ]:
while success:
  cv2.imwrite("./frames/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1
  if count >= 600:
    break

In [ ]:
!mkdir inferred_frame

In [ ]:
# test a single image
for count in range(600):
  img = "./frames/frame%d.jpg" % count
  result = inference_segmentor(model, img)
  show_result_pyplot(model, img, result, get_palette('cityscapes'), out_file = "./inferred_frame/frame%d.jpg" % count)

In [ ]:
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')

out = cv2.VideoWriter('video.avi', fourcc, 25, (854, 480), isColor=True)

for count in range(600):
  img = "./frames/frame%d.jpg" % count
  img = cv2.imread(img)
  out.write(img)
    
out.release()

In [ ]:
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')

out = cv2.VideoWriter('video_inferred.avi', fourcc, 25, (854, 480), isColor=True)

for count in range(600):
  img = "./inferred_frame/frame%d.jpg" % count
  img = cv2.imread(img)
  out.write(img)
    
out.release()